# Introduction

We are given a fictional database which contains data about a music record store called 'Chinook'. The databse contains a schema with 11 tables all connected together.

# Objective

The objective of this project is to answer three questions - 


* Which are the most selling genres of the strore?


* Which countries have the greatest sales performance?


* Should chinook buy full albums or individual tracks from suppliers?

# Connecting to the database

In [1]:
import sqlite3
conn = sqlite3.connect(r'C:\Users\nbnav\OneDrive\Desktop\Dataquest\chinook.db')

# Reading the schema

In [2]:
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
table = cur.fetchall()
print(table)
conn.close()

[('album',), ('artist',), ('customer',), ('employee',), ('genre',), ('invoice',), ('invoice_line',), ('media_type',), ('playlist',), ('playlist_track',), ('track',)]


We can define a function to execute our SQL queries much faster with less lines of code and return the output in the form of a pandas Dataframe.

# Defining a function to execute a query as a pandas dataframe

In [3]:
import pandas as pd
def execute_query(query):
    conn = sqlite3.connect(r'C:\Users\nbnav\OneDrive\Desktop\Dataquest\chinook.db')
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

# Testing our function

In [ ]:
execute_query("SELECT * FROM customer LIMIT 5;")

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


# Finding the most selling genres in the store

In [5]:
execute_query("WITH subquery AS (SELECT * FROM track LEFT JOIN invoice_line ON track.track_id = invoice_line.track_id) SELECT genre.name, COUNT(subquery.genre_id) AS total_sold, CAST(COUNT(subquery.genre_id) AS FLOAT) / SUM(COUNT(subquery.genre_id)) OVER () * 100 AS percentage_sold FROM subquery LEFT JOIN genre ON subquery.genre_id = genre.genre_id GROUP BY genre.name ORDER BY total_sold DESC;")

,name,total_sold,percentage_sold
0,Rock,3017,46.746204
1,Metal,755,11.698172
2,Alternative & Punk,648,10.040285
3,Latin,627,9.714905
4,Jazz,190,2.943911
5,R&B/Soul,165,2.556554
6,Blues,149,2.308646
7,Alternative,123,1.905795
8,Classical,105,1.626898
9,TV Shows,93,1.440967


We can see that 'Rock', 'Metal' and 'Alternative and Punk' genres makes up about 68% of the total sales in chinook music store with 'Rock' alone making 46% of the sales.

# Finding the most profitable countries

In [6]:
execute_query("WITH subquery_2 AS (SELECT * FROM customer LEFT JOIN invoice ON customer.customer_id = invoice.customer_id) SELECT country, SUM(total) AS total_revenue FROM subquery_2 GROUP BY country ORDER BY total_revenue DESC")

,country,total_revenue
0,USA,1040.49
1,Canada,535.59
2,Brazil,427.68
3,France,389.07
4,Germany,334.62
5,Czech Republic,273.24
6,United Kingdom,245.52
7,Portugal,185.13
8,India,183.15
9,Ireland,114.84


We can see that the gross revenue from USA is highest followed by Canada and Brazil, but it does not mean the US market is the most profitable. We should calculate average revenue per customer from each country to determine customers from which country spend the most money on average.

In [14]:
execute_query("WITH subquery_3 AS (SELECT customer.*, SUM(invoice.total) AS total_purchased FROM customer LEFT JOIN invoice ON customer.customer_id = invoice.customer_id GROUP BY customer.customer_id) SELECT country, ROUND(SUM(total_purchased) / COUNT(country), 2) AS average_revenue_per_customer, COUNT(country) AS total_customers FROM subquery_3 GROUP BY country ORDER BY average_revenue_per_customer DESC")

,country,average_revenue_per_customer,total_customers
0,Czech Republic,136.62,2
1,Ireland,114.84,1
2,Spain,98.01,1
3,Chile,97.02,1
4,Portugal,92.57,2
5,India,91.58,2
6,Brazil,85.54,5
7,Germany,83.66,4
8,United Kingdom,81.84,3
9,Australia,81.18,1


The table of average revenue per customer from each country says a different story about the most profitable countries with Czech Republic having the highest revenue per customer followed by Ireland and Spain. We should keep in mind though that the total number of customers in each country is very low to make any concrete assumption.

# Album puchases vs Individual tracks

The Chinook music store plans to determine whether they should purchase whole albums or buy individual tracks based on sales.


In [30]:
execute_query("WITH main_table AS (WITH subquery_4 AS (SELECT invoice_line.*, track.album_id FROM invoice_line JOIN track ON invoice_line.track_id = track.track_id) SELECT invoice_id, album_id, COUNT(album_id) AS total_tracks_purchased FROM subquery_4 GROUP BY 1, 2) SELECT main_table.invoice_id, main_table.album_id, main_table.total_tracks_purchased, CASE WHEN main_table.total_tracks_purchased = album.total_tracks THEN 'Yes' ELSE 'No' END AS full_album_purchased FROM main_table LEFT JOIN (SELECT album_id, COUNT(track_id) AS total_tracks FROM track GROUP BY album_id) AS album ON main_table.album_id = album.album_id")

,invoice_id,album_id,total_tracks_purchased,full_album_purchased
0,1,91,16,Yes
1,2,20,1,No
2,2,34,1,No
3,2,39,1,No
4,2,66,1,No
...,...,...,...,...
3072,613,221,2,No
3073,613,233,1,No
3074,613,257,1,No
3075,613,321,1,No


The above query tells us that for each invoice_id, which albums were purchased and whether were they full album purchases or not. I know the above query looks hella messy but I really suck at commenting my queries and I needed the output as a Pandas Dataframe which prohibited me from indenting my lines. Sorry again but the query works, I checked it manually.

Now we can see how many tracks were purchased as full albums vs how many tracks were purchased as individual tracks.

In [31]:
execute_query("WITH main_table_2 AS (WITH main_table AS (WITH subquery_4 AS (SELECT invoice_line.*, track.album_id FROM invoice_line JOIN track ON invoice_line.track_id = track.track_id) SELECT invoice_id, album_id, COUNT(album_id) AS total_tracks_purchased FROM subquery_4 GROUP BY 1, 2) SELECT main_table.invoice_id, main_table.album_id, main_table.total_tracks_purchased, CASE WHEN main_table.total_tracks_purchased = album.total_tracks THEN 'Yes' ELSE 'No' END AS full_album_purchased FROM main_table LEFT JOIN (SELECT album_id, COUNT(track_id) AS total_tracks FROM track GROUP BY album_id) AS album ON main_table.album_id = album.album_id) SELECT full_album_purchased, SUM(total_tracks_purchased) AS total_tracks_purchased FROM main_table_2 GROUP BY full_album_purchased")

,full_album_purchased,total_tracks_purchased
0,No,3188
1,Yes,1569


We can see that 3188 or 67% of the tracks purchased were individual track purchases while 1569 or 33% of the tracks purchased were full album purchases. Both the numbers seems significant enough to recommend against purchasing only select tracks from albums from record companies.

# Conclusion

So to answer our original three questions - 

* The 'Rock' genre is the most popular genre in terms of sales.

* USA has the highest gross revenue while Czech Republic has the highest Average revenue per customer. The sample data is not big enough to make any concrete assumptions.

* Both individual track purchases and album purchases contribute to the sales figure.